In [281]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

In [282]:
import pandas as pd
import os

In [283]:
files = [file for dirpath, directory, file in os.walk('./all_csv/')][0]
# files=[file for dirpath,directory,file in os.walk(r'/gdrive/My Drive/all_csv/')][0]

In [284]:
match_data = pd.DataFrame(data=None)
odi_scorecard = pd.DataFrame(data=None)
ttwenty_scorecard = pd.DataFrame(data=None)
odi_info = pd.DataFrame(data=None)
ttwenty_info = pd.DataFrame(data=None)

In [285]:
def rename_date_umpire(index_list):
    n = 0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip() == 'date':
            index_list[i] += '_'+str(n)
            n += 1
    n = 0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip() == 'umpire':
            index_list[i] += '_'+str(n)
            n += 1
    n = 0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip() == 'team':
            index_list[i] += '_'+str(n)
            n += 1
    return index_list

In [286]:
def find_game(df_game,df_info):
    if 'series' in df_info.columns:
        if 'odi' in str.lower(df_info.iloc[0]['series']):
            return 'odi'
        if 't20i' in str.lower(df_info.iloc[0]['series']) or 't20' in str.lower(df_info.iloc[0]['series']) or 'indian premier league' in str.lower(df_info.iloc[0]['series'])  or 'indian premier league' in str.lower(df_info.iloc[0]['competition']):
            return 'twenty'
    if max(df_game['balls-bowled'])<=24:
        return 'twenty'
    if 24<max(df_game['balls-bowled'])<=60:
        return 'odi'
    return

In [287]:
def append_file(temp_df, temp_info_df, type_game):
    global odi_scorecard
    global ttwenty_scorecard
    global odi_info
    global ttwenty_info
    if type_game == 'odi':
        odi_scorecard = odi_scorecard.append(temp_df, ignore_index=True)
        odi_info = odi_info.append(temp_info_df, ignore_index=True)
    elif type_game == 'twenty':
        ttwenty_scorecard = ttwenty_scorecard.append(
            temp_df, ignore_index=True)
        ttwenty_info = ttwenty_info.append(temp_info_df, ignore_index=True)

In [288]:
def get_extras_type(match_data):
    list_extras = []
    for index, row in match_data.iterrows():
        ov = str(row['over'])
        if '.' in ov:
            ov = str(row['over']).split('.')
            ball_no = int(ov[1])
            over_no = int(ov[0])
        else:
            continue
        if row['extras'] != 0:
            if row['runs'] != 0:
                match_data.loc[index, 'extras_type'] = 'w'
            list_extras.append(index)
        if ball_no > 6:
            if len(list_extras) > 0:
                match_data.loc[list_extras.pop(-1), 'extras_type'] = 'w'
    for i in list_extras:
        match_data.loc[i, 'extras_type'] = 'b'
    return match_data

In [289]:
def prepare_scorecard(match_data,team_0,team_1):
    match_data = get_extras_type(match_data)
#     print(match_data[match_data['bowler']==	'Mashrafe Mortaza'])
    teams=['','']
    players = list((match_data['striker'].append(
        match_data['non-striker']).append(match_data['bowler'])).unique())
    #     to make 22 players if any player has not played
#     for i in range(len(players),22):
#         players.append('p_'+str(i))
#         fow
#     player_stats=['match-id','innings','name','batting-position','over-batsman','runs-scored','balls-played','ones','twos','threes','fours','sixes','wicket-method','balls-bowled','maiden-overs','runs-given','wickets',extras,'fall-of-wicket-score','fall-of-wicket-overs','fall-of-wicket-no','fow','fow_runs','fow_overs','fow_batsman','fow_bowler']
    player_stats = ['match-id', 'team-name', 'innings', 'name', 'batting-position', 'over-batsman', 'runs-scored', 'balls-played', 'dots', 'ones', 'twos', 'threes', 'fours', 'sixes',
                    'wicket-method', 'balls-bowled', 'maiden-overs', 'runs-given', 'wickets', 'extras', 'fall-of-wicket-score', 'fall-of-wicket-overs', 'fall-of-wicket-no', 'fall-of-wicket-bowler']
    player_data = {key: {key_type: 0 for key_type in player_stats}
                   for key in players}
    for p in players:
        player_data[p]['match-id'] = match_data.loc[0, 'file_no']
        player_data[p]['name'] = p
    team_score = 0
    balls = 0
    pos = 1
    inning = False
    w = 1
    p_no = 1
    w_no = 1
    extras_over = 0
    for index, row in match_data.iterrows():
        ov = str(row['over'])
        if '.' in ov:
            ov = ov.split('.')
            ball_no = int(ov[1])
            over_no = int(ov[0])
        else:
            continue
        if over_no > 50:
            player_data = [value for key, value in player_data.items()]
            scorecard = pd.DataFrame(data=player_data)
            scorecard = scorecard[player_stats]
            return scorecard
        if ball_no == 1 and over_no == 0:
            pos = 1
            w = 1
            team_score = 0
            w_no = 1
            runs_over = 0
            if row['innings']==1 and teams[0]=='':
                teams[0]=row['batting-team']
                if teams[0]==team_0:
                    teams[1]=team_1
                elif teams[0]==team_1:
                    teams[1]=team_0
            # if row['innings']!=1:
            #     p_no=12
        if ball_no == 1:
            extras_over = 0
        if row['runs'] == 1:
            player_data[row['striker']]['ones'] += 1
        elif row['runs'] == 2:
            player_data[row['striker']]['twos'] += 1
        elif row['runs'] == 3:
            player_data[row['striker']]['threes'] += 1
        elif row['runs'] == 4:
            player_data[row['striker']]['fours'] += 1
        elif row['runs'] == 6:
            player_data[row['striker']]['sixes'] += 1
        elif row['extras'] == 0:
            player_data[row['striker']]['dots'] += 1
        if player_data[row['striker']]['batting-position'] == 0:
            player_data[row['striker']]['batting-position'] = pos
#             print(type(row['over']),type(extras_over))
#             print(row['over'])
            player_data[row['striker']
                        ]['over-batsman'] = float(row['over'])-extras_over
            pos += 1
        if player_data[row['non-striker']]['batting-position'] == 0:
            player_data[row['non-striker']]['batting-position'] = pos
            player_data[row['non-striker']
                        ]['over-batsman'] = float(row['over'])-extras_over
            pos += 1
    # wicket
#         print(row['out-player'])
        if not pd.isna(row['out-player']):
            player_data[row['out-player']]['wicket-method'] = row['out']
#             fow
#             player_data[players[p_no-1]]['fow']=w
#             player_data[players[p_no-1]]['fow_runs']=team_score
#             player_data[players[p_no-1]]['fow_overs']=row['over']
#             player_data[players[p_no-1]]['fow_batsman']=row['out-player']
#             player_data[players[p_no-1]]['fow_bowler']=row['bowler']
            p_no += 1
            w += 1
            if row['out'] != 'run out':
                player_data[row['bowler']]['wickets'] += 1
            player_data[row['out-player']]['fall-of-wicket-score'] = team_score
            player_data[row['out-player']
                        ]['fall-of-wicket-overs'] = float(row['over'])-extras_over
            player_data[row['out-player']]['fall-of-wicket-no'] = w_no
            player_data[row['out-player']
                        ]['fall-of-wicket-bowler'] = row['bowler']
            w_no += 1
        team_score += row['runs']+row['extras']
        runs_over += row['runs']
        if row['extras'] != 0 and row['extras_type'] == 'w':
            runs_over += 1
            player_data[row['bowler']]['extras'] += 1
            player_data[row['striker']]['runs-scored'] += row['extras']-1
            player_data[row['bowler']]['runs-given'] += row['extras']-1
            extras_over += 0.1
        elif row['extras'] != 0:
            #             print(row)
            player_data[row['bowler']]['balls-bowled'] += 1
            player_data[row['striker']]['balls-played'] += 1
        else:
            player_data[row['striker']]['balls-played'] += 1
            player_data[row['bowler']]['balls-bowled'] += 1
            player_data[row['bowler']]['runs-given'] += row['runs']
        player_data[row['striker']]['runs-scored'] += row['runs']
        if ball_no >= 6:
            if ball_no == 6 and runs_over == 0:
                player_data[row['bowler']]['maiden-overs'] += 1
            runs_over = 0
        player_data[row['striker']]['innings'] = row['innings']
        player_data[row['striker']]['team-name']=row['batting-team']
        player_data[row['non-striker']]['team-name']=row['batting-team']
        if row['innings'] == 1:
            player_data[row['bowler']]['innings'] = 2
            player_data[row['bowler']]['team-name']=teams[1]
#             print(teams,row['bowler'])
        elif row['innings']==2:
            player_data[row['bowler']]['innings'] = 1
            player_data[row['bowler']]['team-name']=teams[0]
#             print(teams,row['bowler'])
#     print(player_data)
    
    player_data = [value for key, value in player_data.items()]
    scorecard = pd.DataFrame(data=player_data)
    scorecard = scorecard[player_stats]
    return scorecard

In [290]:
i = 0
for file in files[:5]:
    print(i, '--', file)
    i += 1
    count = 0
    df_index = []
    df_row = []
    add = r'./all_csv/'+file
#     add=r'/gdrive/My Drive/all_csv/'+file
    df_index = ['file_no']
    file_no = file.split('.')[0]
    df_row = [file_no]
    with open(add) as f:
        new_f = f.readlines()
        for line in new_f:
            if 'version' in line:
                count += 1
            elif 'info' in line:
                count += 1
                line = line.strip().split(',')
                df_index.append(line[1])
                df_row.append(line[2])
            else:
                df_index = rename_date_umpire(df_index)
                df_dic = dict(zip(df_index, df_row), index=[0])
                temp_info_df = pd.DataFrame(df_dic)
#                 df_info=df_info.append(temp_info_df,ignore_index=True)
                # gender=df_info['gender'].iloc[0]
                # gender=str.lower(gender.strip())
                break
    temp_df = pd.read_csv(add, skiprows=count, names=[
                          0, 'innings', 'over', 'batting-team', 'striker', 'non-striker', 'bowler', 'runs', 'extras', 'out', 'out-player'])
    temp_df = temp_df.drop([0], axis=1)
    temp_df['file_no'] = [file_no]*(temp_df.shape[0])
    temp_sc = prepare_scorecard(temp_df,temp_info_df['team_0'].values[0],temp_info_df['team_1'].values[0])
    # print(temp_sc)
#     append_file(temp_df,gender,type_game)
    append_file(temp_sc, temp_info_df, find_game(temp_sc, temp_info_df))

0 -- 1019975.csv
1 -- 682919.csv
2 -- 952191.csv
3 -- 1043961.csv
4 -- 565820.csv


In [291]:
odi_info.columns

Index(['city', 'competition', 'date_0', 'file_no', 'gender', 'index',
       'match_number', 'match_referee', 'player_of_match', 'reserve_umpire',
       'season', 'series', 'team_0', 'team_1', 'toss_decision', 'toss_winner',
       'tv_umpire', 'umpire_0', 'umpire_1', 'venue', 'winner', 'winner_runs'],
      dtype='object')

In [292]:
ttwenty_info.columns

Index(['city', 'competition', 'date_0', 'file_no', 'gender', 'index',
       'match_number', 'match_referee', 'neutralvenue', 'player_of_match',
       'reserve_umpire', 'season', 'series', 'team_0', 'team_1',
       'toss_decision', 'toss_winner', 'tv_umpire', 'umpire_0', 'umpire_1',
       'venue', 'winner', 'winner_runs', 'winner_wickets'],
      dtype='object')

In [293]:
ttwenty_info

,city,competition,date_0,file_no,gender,index,match_number,match_referee,neutralvenue,player_of_match,...,team_1,toss_decision,toss_winner,tv_umpire,umpire_0,umpire_1,venue,winner,winner_runs,winner_wickets
0,,World T20,2014/03/21,682919,male,0,12,DC Boon,true,SJ Myburgh,...,Netherlands,field,Netherlands,BF Bowden,SJ Davis,IJ Gould,Sylhet Stadium,Netherlands,NaN,6
1,Chennai,NaN,2012/09/11,565820,male,0,2,BC Broad,NaN,BB McCullum,...,New Zealand,field,India,S Asnani,VA Kulkarni,S Ravi,"""MA Chidambaram Stadium",New Zealand,1,NaN


In [294]:
ttwenty_scorecard

,match-id,team-name,innings,name,batting-position,over-batsman,runs-scored,balls-played,dots,ones,...,wicket-method,balls-bowled,maiden-overs,runs-given,wickets,extras,fall-of-wicket-score,fall-of-wicket-overs,fall-of-wicket-no,fall-of-wicket-bowler
0,682919,Ireland,1,WTS Porterfield,1,0.1,47,32,13,9,...,bowled,0,0,0,0,0,69,9.1,2,Ahsan Malik
1,682919,Ireland,1,PR Stirling,2,0.1,7,9,6,1,...,caught,12,0,24,0,0,28,3.2,1,T van der Gugten
2,682919,Ireland,1,EC Joyce,3,3.3,28,25,6,14,...,run out,0,0,0,0,0,88,12.3,3,Mudassar Bukhari
3,682919,Ireland,1,AD Poynter,4,9.2,57,38,12,15,...,caught,0,0,0,0,0,189,19.6,4,Ahsan Malik
4,682919,Ireland,1,KJ O'Brien,5,12.4,42,16,1,8,...,0,18,0,29,2,0,0,0.0,0,0
5,682919,Netherlands,2,PW Borren,1,0.1,31,15,4,3,...,caught,12,0,18,0,1,91,5.6,1,KJ O'Brien
6,682919,Netherlands,2,SJ Myburgh,2,0.1,63,23,7,3,...,caught,0,0,0,0,0,98,6.3,2,GH Dockrell
7,682919,Netherlands,2,W Barresi,3,6.1,40,22,6,10,...,0,0,0,0,0,0,0,0.0,0,0
8,682919,Netherlands,2,LV van Beek,4,6.4,1,3,2,1,...,caught,6,0,18,0,0,100,7.2,3,KJ O'Brien
9,682919,Netherlands,2,TLW Cooper,5,7.3,45,15,3,3,...,caught,6,0,8,0,0,161,11.2,4,TJ Murtagh


In [295]:
odi_info

,city,competition,date_0,file_no,gender,index,match_number,match_referee,player_of_match,reserve_umpire,...,team_0,team_1,toss_decision,toss_winner,tv_umpire,umpire_0,umpire_1,venue,winner,winner_runs
0,Nelson,NaN,2016/12/29,1019975,male,0,2,BC Broad,NT Broom,WR Knights,...,New Zealand,Bangladesh,field,Bangladesh,C Shamshuddin,CM Brown,PR Reiffel,Saxton Oval,New Zealand,67
1,Abu Dhabi,"""ICC World Cricket League Championship",2016/01/29,952191,male,0,20,DT Jukes,NaN,Rabiul Hoque,...,United Arab Emirates,Netherlands,field,United Arab Emirates,NaN,CK Nandan,Sarika Prasad,Sheikh Zayed Stadium,Netherlands,6
2,Coffs Harbour,NaN,2016/11/29,1043961,female,0,5,PL Marshall,EA Perry,DJ Shepard,...,Australia,South Africa,field,South Africa,NaN,PJ Gillespie,JD Ward,International Sports Stadium,Australia,43


In [296]:
odi_scorecard

,match-id,team-name,innings,name,batting-position,over-batsman,runs-scored,balls-played,dots,ones,...,wicket-method,balls-bowled,maiden-overs,runs-given,wickets,extras,fall-of-wicket-score,fall-of-wicket-overs,fall-of-wicket-no,fall-of-wicket-bowler
0,1019975,New Zealand,1,MJ Guptill,1,0.1,0,4,4,0,...,lbw,0,0,0,0,0,0,0.4,1,Mashrafe Mortaza
1,1019975,New Zealand,1,KS Williamson,3,0.5,14,35,27,4,...,caught,30,0,21,3,1,37,10.5,2,Taskin Ahmed
2,1019975,New Zealand,1,TWM Latham,2,0.1,22,35,25,4,...,lbw,0,0,0,0,0,47,13.1,3,Shakib Al Hasan
3,1019975,New Zealand,1,NT Broom,4,10.6,109,107,42,49,...,0,0,0,0,0,0,0,0.0,0,0
4,1019975,New Zealand,1,JDS Neesham,5,13.2,28,31,17,10,...,stumped,12,0,12,0,1,98,22.5,4,Mosaddek Hossain
5,1019975,New Zealand,1,C Munro,6,22.6,3,7,4,3,...,bowled,12,0,12,0,0,107,25.3,5,Mashrafe Mortaza
6,1019975,New Zealand,1,L Ronchi,7,25.4,35,38,21,11,...,caught,0,0,0,0,0,171,37.3,6,Taskin Ahmed
7,1019975,New Zealand,1,MJ Santner,8,37.4,9,17,12,3,...,caught,48,1,19,1,1,198,42.5,7,Subashis Roy
8,1019975,New Zealand,1,TG Southee,9,42.6,3,7,5,1,...,caught,54,1,31,2,2,214,45.2,8,Shakib Al Hasan
9,1019975,New Zealand,1,LH Ferguson,10,45.3,4,7,5,0,...,caught,48,0,50,1,3,228,46.5,9,Mashrafe Mortaza


In [297]:
odi_scorecard['fall-of-wicket-overs'].value_counts()

0.0     7
45.2    3
42.5    2
44.4    2
49.3    2
48.5    2
25.3    2
11.4    1
47.1    1
49.6    1
36.4    1
32.4    1
20.6    1
14.4    1
13.1    1
1.6     1
47.5    1
43.6    1
29.1    1
42.1    1
30.5    1
25.5    1
37.5    1
9.5     1
6.5     1
46.5    1
22.5    1
49.4    1
37.2    1
10.1    1
32.2    1
48.3    1
43.3    1
32.2    1
17.4    1
41.1    1
7.6     1
2.3     1
39.3    1
26.3    1
30.4    1
23.1    1
28.4    1
22.6    1
12.6    1
0.4     1
37.3    1
31.3    1
47.4    1
35.2    1
46.4    1
7.2     1
10.5    1
Name: fall-of-wicket-overs, dtype: int64

In [280]:
odi_info.to_csv('./odi_info.csv', index=False)
ttwenty_info.to_csv('./ttwenty_info.csv', index=False)
odi_scorecard.to_csv('./odi_scorecard.csv', index=False)
ttwenty_scorecard.to_csv('./ttwenty_scorecard.csv', index=False)